In [26]:
import pandas as pd

df = pd.read_feather('/workspace/data/hn/stories-tokenized.feather')
df.columns


Index(['by', 'comments', 'id', 'score', 'time', 'title', 'type', 'url', 'dead',
       'text', 'kids', 'deleted', 'frontpage', 'formatted_text', 'input_ids',
       'token_type_ids', 'attention_mask'],
      dtype='object')

In [28]:
df = df.sort_values('time')
split_date = df.iloc[int(len(df) * 0.95)]['time']

split_date

df['split'] = 'train'
df['split'].loc[df['time'] > split_date] = 'test'

/tmp/ipykernel_6539/466835916.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['split'].loc[df['time'] > split_date] = 'test'


In [30]:
# See the % of stories that hit the front page broken down by split

df.groupby('split').frontpage.mean()

split
test     0.170221
train    0.126423
Name: frontpage, dtype: float64

In [ ]:
split_date

In [23]:
df = df[['id', 'by', 'comments', 'score', 'time', 'title', 'url', 'frontpage', 'input_ids']]

KeyError: "['input_ids'] not in index"

In [6]:
df.frontpage.value_counts()

frontpage
False    1323006
True      195270
Name: count, dtype: int64

In [9]:
# Get the top authors by percentage of frontpage stories
# df.groupby

# limit to folks who have submitted at least 3 stories that made the front page
prolific_submitters = df[df.frontpage == True].by.value_counts() > 3

new_df = df[df.by.isin(prolific_submitters[prolific_submitters].index)]

new_df.shape

(891739, 8)

In [16]:
df['prolific'] = df.by.isin(prolific_submitters[prolific_submitters].index)

df['fraction_on_frontpage'] = df.groupby('by').frontpage.transform('mean')

df[df.prolific & (df.fraction_on_frontpage > 0.5)]

,id,by,comments,score,time,title,url,frontpage,prolific,fraction_fp,fraction_on_frontpage
18,25015967,granzymes,4576.0,3089.0,2020-11-07 16:25:47,"Biden wins White House, vowing new direction f...",https://apnews.com/article/joe-biden-wins-whit...,True,True,NaN,0.571429
66,19632449,kragniz,1119.0,2369.0,2019-04-11 09:37:56,Julian Assange arrested in London,https://www.bbc.co.uk/news/uk-47891737,True,True,NaN,0.555556
124,36283249,Freddie111,1133.0,1985.0,2023-06-11 17:10:10,Reddit Strike Has Started,https://reddark.untone.uk/,True,True,NaN,0.571429
145,25033844,isp,1148.0,1887.0,2020-11-09 11:59:05,Covid vaccine: First ‘milestone’ vaccine offer...,https://www.bbc.co.uk/news/health-54873105,True,True,NaN,0.542857
153,17022963,ivank,750.0,1875.0,2018-05-08 17:51:44,Google Duplex: An AI System for Accomplishing ...,https://ai.googleblog.com/2018/05/duplex-ai-sy...,True,True,NaN,0.689655
...,...,...,...,...,...,...,...,...,...,...,...
1517148,21210806,domino,0.0,1.0,2019-10-10 04:11:37,Serverless State: ServerlessConf Presentation ...,https://docs.google.com/presentation/d/1g06Umz...,False,True,NaN,0.666667
1517625,27503831,BCM43,0.0,1.0,2021-06-14 15:33:53,Stronger data security and privacy with Google...,https://workspaceupdates.googleblog.com/2021/0...,False,True,NaN,0.555556
1517849,27521590,Petiver,0.0,1.0,2021-06-15 21:14:36,Joseph George Strutt's Sylva Britannica (1822/...,https://publicdomainreview.org/collection/sylv...,False,True,NaN,0.707819
1518102,21206261,prismatic,0.0,1.0,2019-10-09 18:24:40,Chinese Influence in Persian Manuscript Illust...,http://ijmcr.com/wp-content/uploads/2017/08/Pa...,False,True,NaN,0.594883


In [22]:
df['fp_guess'] = 0

df.loc[df.prolific & (df.fraction_on_frontpage > 0.5), 'fp_guess'] = 1

# determine the MSE between the guess and the actual

mse = ((df.frontpage.astype(float) - df.fp_guess)**2).mean()
mse

0.12542515326594111

In [25]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'[CLS] Title: Stephen Hawking has died URL: http://www.bbc.com/news/uk-43396008 Poster: Cogito Date: Wednesday, March 14, 03:50 AM[SEP]'

In [ ]:

print('tokenizer output')
tokenizer.decode(df.input_ids[0])
print(df.formatted_text[0])